# part 2 | learn Binning and Binarization

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import Binarizer
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv('/kaggle/input/train-data/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df = df[['Age', 'Fare', 'SibSp', 'Parch', 'Survived']]

In [6]:
df.dropna(inplace= True)

In [7]:
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [8]:
df['family'] = df['SibSp'] + df['Parch']
df.head()

,Age,Fare,SibSp,Parch,Survived,family
0,22.0,7.2500,1,0,0,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [9]:
df.drop(columns= ['SibSp', 'Parch'], inplace= True)

In [10]:
df.sample(5)

,Age,Fare,Survived,family
13,39.0,31.2750,0,6
509,26.0,56.4958,1,0
447,34.0,26.5500,1,0
657,32.0,15.5000,0,2
567,29.0,21.0750,0,4


In [11]:
X= df.drop(columns= ['Survived'])
y= df['Survived']

In [12]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 42)

In [13]:
X_train.shape

(571, 3)

In [14]:
dt= DecisionTreeClassifier(random_state= 0)
dt.fit(X_train, y_train)
y_pred= dt.predict(X_test)

accuracy_score(y_test, y_pred)

0.6083916083916084

In [15]:
np.mean(cross_val_score(dt, X,y, cv= 10, scoring= 'accuracy'))

0.6541471048513302

In [16]:
CT= ColumnTransformer([
    ('bin', Binarizer(copy= False),['family'])
], remainder= 'passthrough')

In [17]:
X_train_CT= CT.fit_transform(X_train)
X_test_CT= CT.transform(X_test)

In [18]:
pd.DataFrame(X_train_CT, columns= ['family', 'Age', 'Fare'])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [19]:
dt= DecisionTreeClassifier(random_state= 0)
dt.fit(X_train_CT, y_train)
y_pred2= dt.predict(X_test_CT)

accuracy_score(y_test, y_pred2)

0.6153846153846154

In [20]:
X_CT= CT.fit_transform(X)
np.mean(cross_val_score(dt, X_CT,y, cv=10, scoring= 'accuracy'))

0.6261737089201878